In [1]:
# import csv
import keras
#from keras.optimizers import *
import pandas as pd
# from sklearn.svm import SVC
import tensorflow as tf
# from plaidml.tile import ShapeOf
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
# from sklearn.preprocessing import StandardScaler
# from multiprocessing import Pool
from sklearn.model_selection import train_test_split
# from tensorflow.metrics import *

Using plaidml.keras.backend backend.


In [2]:
def gal_loss(true, pred):
    true_sum = K.sum(true, axis = 0)
    true_zero = K.zeros_like(true_sum)
    alt = K.ones_like(true) - true
#     print(K.eval(alt))
    alt_sum = K.sum(alt)
    alt_zero = K.zeros_like(alt_sum)
    alt_pred = K.ones_like(pred) - pred
    tpr = (K.sum(true * pred)+1)/(K.sum(true)+1)*100
    tnr = (K.sum(alt * alt_pred)+1)/(K.sum(alt)+1)*100
    mu = 20000-(tpr*tnr-K.pow((tpr-tnr), 2)+10000)
#     mu = 2*10**6-(tpr*tnr-(tpr-tnr)**2+10**6)
    return mu

In [3]:
def to_int_map(c):
    if c == 'A':
        return 0
    elif c == 'C':
        return 1
    elif c == 'D':
        return 2
    elif c == 'G':
        return 3
    elif c == 'N':
        return 4
    elif c == 'R':
        return 5
    elif c == 'S':
        return 6
    elif c == 'T':
        return 7
    else:
        print('ERROR!')
    return c
def solutions_map(c):
    #EI, IE, N
    if c == 'EI':
        return 0
    elif c == 'IE':
        return 1
    elif c == "N":
        return 2
    else:
        print("ERROR")
    return c
train_data = pd.read_csv('splice.data', header=None)
solutions = train_data[0]
# train_data = train_data[0] + train_data[1]
train_data = train_data[2]
# print(train_data)
t = []
# test = []
for sample in train_data:
    t += [[to_int_map(l) for l in ''.join(sample.split())]]
from keras.utils import to_categorical
# t = to_categorical(t)
# temp = []
# for temp1 in t:
#     temp3 = []
#     for temp2 in temp1:
# #         temp4 = []
#         for temp5 in temp2:
#             temp3 += [temp5]
# #             print(temp5)
# #         print(temp4)
# #     print(len(temp3))
#     temp += [temp3]
# # t = [[l for l in m for m in k] for k in t]
# # print(t)
# t = temp
# solutions = to_categorical(solutions)
# print(solutions)
# t = pd.DataFrame(t)
# solutions = pd.DataFrame(solutions)
# print(solutions)
# train_features, test_features, train_solutions, test_solutions = train_test_split(t, solutions
#                                                                                   , test_size=0.01
#                                                                                  )
# print(train_solutions)
# print(train_features)
# print(train_solutions)
import numpy as np
# print(K.eval(gal_loss(np.array(train_solutions), np.array(test_solutions))))

In [4]:
# from sklearn.model_selection import StratifiedKFold
# kfold = StratifiedKFold(n_splits=10, shuffle=True)
# cvscores = []
# for train, test in kfold.split(t, solutions):
#     s_temp = [solutions_map(tempo) for tempo in solutions]
#     s_temp = to_categorical(s_temp)
#     t_temp = to_categorical(t)
#     temp = []
#     for temp1 in t_temp:
#         temp3 = []
#         for temp2 in temp1:
#             for temp5 in temp2:
#                 temp3 += [int(temp5)]
#         temp += [temp3]
#     t_temp = temp
#     model = Sequential()
#     nodes = 13
#     model.add(Dense(50, input_dim=480, activation='relu'))
# #     while nodes > 1:
# #         model.add(Dense(nodes, activation='relu'))
# #         if nodes % 2 == 0:
# #             nodes = int(nodes/2)
# #         else:
# #             nodes = 3 * nodes + 1
#     model.add(Dense(3, activation='sigmoid'))
#     # Compile model
#     model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#     # Fit the model
#     t_temp = np.array(t_temp)
#     s_temp = np.array(s_temp)
#     model.fit(t_temp[train], s_temp[train], epochs=150, batch_size=2048, verbose=0)
#     # evaluate the model
#     scores = model.evaluate(t_temp[test], s_temp[test], verbose=0)
#     print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
#     cvscores.append(scores[1] * 100)
# print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

In [7]:
# s_temp = [solutions_map(tempo) for tempo in solutions]
# s_temp = to_categorical(s_temp)
# t_temp = to_categorical(t)
# # print(t_temp)
# temp = []
# for temp1 in t_temp:
#     temp3 = []
#     for temp2 in temp1:
#         for temp5 in temp2:
#             temp3 += [int(temp5)]
#     temp += [temp3]
# t_temp = temp
# print(t_temp[0])
train_features, test_features, train_solutions, test_solutions = train_test_split(t, solutions, test_size=0.7)
# train_solutions = [list(l) for l in train_solutions]
# print(train_solutions[0])

from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=10, shuffle=True)
cvscores = []
for train, test in kfold.split(train_features, train_solutions):
    s_temp = [solutions_map(tempo) for tempo in train_solutions]
    s_temp = to_categorical(s_temp)
    t_temp = to_categorical(train_features)
    temp = []
    for temp1 in t_temp:
        temp3 = []
        for temp2 in temp1:
            for temp5 in temp2:
                temp3 += [int(temp5)]
        temp += [temp3]
    t_temp = temp
    model = Sequential()
    model.add(Dense(50, input_dim=480, activation='relu'))
    model.add(Dense(3, activation='sigmoid'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit the model
    t_temp = np.array(t_temp)
    s_temp = np.array(s_temp)
    model.fit(t_temp[train], s_temp[train], epochs=150, batch_size=2048, verbose=0)
    # evaluate the model
    scores = model.evaluate(t_temp[test], s_temp[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

model = Sequential()
nodes = 13
model.add(Dense(50, input_dim=480, activation='relu'))
model.add(Dense(3, activation='sigmoid'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
train_features = np.array(train_features)
test_features = np.array(test_features)
# print(train_features[0])
# print("---")
# print(train_solutions[0])
model.fit(train_features, train_solutions, epochs=150, batch_size=2048, verbose=0)
# evaluate the model
scores = model.evaluate(test_features, test_solutions, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
train_acc = model.evaluate(train_features, test_features, verbose = 0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

acc: 95.88%
acc: 91.75%
acc: 90.72%
acc: 95.88%
acc: 94.79%
acc: 90.53%
acc: 97.89%
acc: 95.79%
acc: 92.55%
acc: 91.49%
93.73% (+/- 2.48%)


ValueError: Error when checking input: expected dense_65_input to have shape (480,) but got array with shape (60,)